In [48]:
import warnings
import requests
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime, timedelta
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [49]:
def fetch_historical_data(index, expiry_date):
    url = f"https://live.markethound.in/api/history/decay?name={index}&expiry={expiry_date}&dte=0"
    response = requests.get(url)
    data = response.json()
    return data['result']

In [50]:
def preprocess_data(data):
    df = pd.DataFrame(data)
    df['date'] = pd.to_datetime(df['date'])
    numerical_columns = ['close', 'high', 'low', 'open', 'intradayMovement', 'intradayTotal']
    df[numerical_columns] = df[numerical_columns].astype(float)
    df.sort_values(by='date', inplace=True)
    return df

In [52]:
def train_models(df):
    X = df[['index_open', 'index_high', 'index_low', 'index_close']]
    y = df[['open', 'high', 'low', 'close']]

    models = {
        'Linear Regression': LinearRegression(),
        'Decision Tree': DecisionTreeRegressor(),
        'Random Forest': RandomForestRegressor()
    }

    trained_models = {}
    for name, model in models.items():
        model.fit(X, y)
        trained_models[name] = model

    return trained_models

In [53]:
def make_predictions(models, index_data):
    predictions = {}
    for name, model in models.items():
        next_day_features = index_data[['index_open', 'index_high', 'index_low', 'index_close']]
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            prediction = model.predict(next_day_features)
        prediction = prediction.reshape(1, -1)
        predictions[name] = prediction

    return predictions


In [54]:
def main():
    expiry_url = "https://live.markethound.in/api/history/expiries?index=FINNIFTY"
    expiry_response = requests.get(expiry_url)
    expiry_data = expiry_response.json()['result']

    for expiry_date in expiry_data:
        data = fetch_historical_data("FINNIFTY", expiry_date)
        if not data:
            continue

        df = preprocess_data(data)

        models = train_models(df)

        next_day = datetime.strptime(expiry_date, "%Y-%m-%dT%H:%M:%S.%fZ") + timedelta(days=1)
        index_data_next_day = fetch_historical_data("FINNIFTY", next_day.strftime("%Y-%m-%dT%H:%M:%S.%fZ"))
        if not index_data_next_day:
            continue
        index_data_next_day_df = preprocess_data(index_data_next_day)  # Preprocess data
        predictions = make_predictions(models, index_data_next_day_df)

        mae_dict = {}
        for model_name, prediction in predictions.items():
            true_values = index_data_next_day_df[['open', 'high', 'low', 'close']].values
            prediction = prediction.reshape(true_values.shape)
            mae = mean_absolute_error(true_values, prediction)
            mae_dict[model_name] = mae
        best_model = min(mae_dict, key=mae_dict.get)
        # r2_dict = {}
        # for model_name, prediction in predictions.items():
        #     true_values = index_data_next_day_df[['open', 'high', 'low', 'close']].values
        #     prediction = prediction.reshape(true_values.shape)
        #     r2 = r2_score(true_values, prediction)
        #     r2_dict[model_name] = r2

        # best_model = max(r2_dict, key=r2_dict.get)

        print("Expiry Date:", expiry_date)
        print("Predicted Straddle Prices for Next Day:")
        for model_name, prediction in predictions.items():
            print(f"Model: {model_name}")
            print("Open:", prediction[0][0])
            print("High:", prediction[0][1])
            print("Low:", prediction[0][2])
            print("Close:", prediction[0][3])
            print()
        print("Best Model:", best_model)
        print()

In [55]:
if __name__ == "__main__":
    main()

Expiry Date: 2024-03-12T00:00:00.000Z
Predicted Straddle Prices for Next Day:
Model: Linear Regression
Open: 121.39999999999975
High: 132.84999999999934
Low: 5.350000000000051
Close: 9.899999999999835

Model: Decision Tree
Open: 121.4
High: 132.85
Low: 5.35
Close: 9.9

Model: Random Forest
Open: 123.75349999999985
High: 137.99950000000024
Low: 4.211000000000005
Close: 7.381499999999992

Best Model: Decision Tree

Expiry Date: 2024-03-05T00:00:00.000Z
Predicted Straddle Prices for Next Day:
Model: Linear Regression
Open: 148.5499999999996
High: 148.54999999999833
Low: 9.149999999999409
Close: 15.350000000000307

Model: Decision Tree
Open: 148.55
High: 148.55
Low: 9.15
Close: 15.35

Model: Random Forest
Open: 139.21899999999982
High: 146.14799999999994
Low: 6.969999999999992
Close: 11.630000000000004

Best Model: Decision Tree

Expiry Date: 2024-02-27T00:00:00.000Z
Predicted Straddle Prices for Next Day:
Model: Linear Regression
Open: 163.50000000000057
High: 183.2500000000008
Low: 9.749